In [260]:
import re
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

from os import listdir
from os.path import isfile, join

In [261]:
INPUT_PATH = f"out/"
OUTPUT_PATH = "clean_out/"
FILES = onlyfiles = [f for f in listdir(INPUT_PATH) if isfile(join(INPUT_PATH, f))]
FILES

['1-s2.0-S0166354216304016-main.tei.xml',
 'PIIS0140673620303743.tei.xml',
 'radiol.2020200370.tei.xml',
 '1-s2.0-S0163445313000716-main.tei.xml',
 'PIIS1473309920301201.tei.xml',
 'nejmp2002125.tei.xml',
 'NEJM2013.4.23-111H7N9cases.tei.xml',
 'jcm-09-00523-v2.tei.xml',
 'PIIS0140673620303652.tei.xml',
 'bmj.m641.full.tei.xml',
 '1-s2.0-S0163445313001114-main.tei.xml',
 'PIIS221326002030076X.tei.xml',
 '1-s2.0-S0006291X14013321-main.tei.xml',
 '1-s2.0-S0140673620304116-main.tei.xml',
 '1-s2.0-S1473309920300864-main.tei.xml',
 'aqaa029.tei.xml',
 'jkms-35-e79.tei.xml',
 'CORONAVIRUS.tei.xml',
 'nejmoa030781.tei.xml',
 '1-s2.0-S0166354216301917-main.tei.xml',
 '1-s2.0-S0042682219301631-main.tei.xml',
 'ciaa247.tei.xml',
 '1-s2.0-S1090023317300059-main.tei.xml',
 'jama_bai_2020_ld_200013.tei.xml',
 '1-s2.0-S0065352716300458-main.tei.xml',
 'advpub_2020.01047.tei.xml',
 'Stuerzlinger2006_Article_TheDesignAndRealizationOfCoViD.tei.xml',
 '4240.full.tei.xml',
 'jama_wu_2020_vp_200028.tei.xm

## Remove Tags
 - tag ref type=bibr
 - tag ref type=figure
 - remover tag figure e manter tag figDesc, dentro de figure
 - remover tag ref type=table
 - remover tag table 
 - remover tag formula

In [262]:
def get_node(parent):
    if parent:
        rem_child = []
        add_child = []
        for child in parent:
            if child.tag == namespace(root)+"figure":
                figDesc = child.find(namespace(root)+'figDesc')
                add_child.append(figDesc)
                rem_child.append(child)
            if child.tag in [namespace(root)+"table", namespace(root)+"formula"]:
                rem_child.append(child)
            elif child.tag == namespace(root)+"ref":
#                 print(child.attrib)
                if 'type' in child.attrib and child.attrib['type'] in ['bibr', 'figure', 'table']:
                    rem_child.append(child)
        for child in rem_child:
            parent.remove(child)
        for child in add_child:
            parent.append(child)
        for child in parent:
            yield from get_node(child)

In [263]:
def namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    return m.group(0) if m else ''

In [264]:
ET.register_namespace('', "http://www.tei-c.org/ns/1.0")
ET.register_namespace('ows', "http://www.opengis.net/ows/1.1")
ET.register_namespace('xlink', "http://www.w3.org/1999/xlink")
ET.register_namespace('xsi', "http://www.w3.org/2001/XMLSchema-instance")


## Init

In [268]:
for curr_file in FILES:
    tree = ET.parse(INPUT_PATH + curr_file)
    root = tree.getroot()
    list(get_node(root))
    #get_node(root)
    tree.write(OUTPUT_PATH + curr_file)

In [267]:
# Write XML file
rough_string = ET.tostring(root, 'UTF-8')
rough_string = rough_string.replace(b'\t', b'')
rough_string = rough_string.replace(b'\n', b'')
reparsed = minidom.parseString(rough_string)
#xml_file = open(OUTPUT_PATH + curr_file.split('.')[0]+'-'+str(it)+'.xml', 'w')
xml_file = open('clean_out/1-s2.0-S0006291X03024136-main.tei.xml', 'w')
xml_file.write(reparsed.toprettyxml(indent="\t", encoding='UTF-8').decode("utf-8"))
xml_file.close()